In [68]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [69]:
train = pd.read_parquet("train.parquet")
test = pd.read_parquet("test.parquet")

train.shape, test.shape

((17499636, 19), (4393179, 19))

In [70]:
train.head()

,status,gender,firstName,level,lastName,userId,ts,auth,page,sessionId,location,itemInSession,userAgent,method,length,song,artist,time,registration
0,200,M,Shlok,paid,Johnson,1749042,1538352001000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",278,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,524.32934,Ich mache einen Spiegel - Dream Part 4,Popol Vuh,2018-10-01 00:00:01,2018-08-08 13:22:21
992,200,M,Shlok,paid,Johnson,1749042,1538352525000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",279,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,178.02404,Monster (Album Version),Skillet,2018-10-01 00:08:45,2018-08-08 13:22:21
1360,200,M,Shlok,paid,Johnson,1749042,1538352703000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",280,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,232.61995,Seven Nation Army,The White Stripes,2018-10-01 00:11:43,2018-08-08 13:22:21
1825,200,M,Shlok,paid,Johnson,1749042,1538352935000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",281,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,265.50812,Under The Bridge (Album Version),Red Hot Chili Peppers,2018-10-01 00:15:35,2018-08-08 13:22:21
2366,200,M,Shlok,paid,Johnson,1749042,1538353200000,Logged In,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",282,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",PUT,471.69261,Circlesong 6,Bobby McFerrin,2018-10-01 00:20:00,2018-08-08 13:22:21


In [71]:
train.columns

Index(['status', 'gender', 'firstName', 'level', 'lastName', 'userId', 'ts',
       'auth', 'page', 'sessionId', 'location', 'itemInSession', 'userAgent',
       'method', 'length', 'song', 'artist', 'time', 'registration'],
      dtype='object')

In [72]:
train.dtypes

status                    int64
gender                   object
firstName                object
level                    object
lastName                 object
userId                   object
ts                        int64
auth                     object
page                     object
sessionId                 int64
location                 object
itemInSession             int64
userAgent                object
method                   object
length                  float64
song                     object
artist                   object
time             datetime64[us]
registration     datetime64[us]
dtype: object

In [114]:
train['time'].min(), train['time'].max()

(Timestamp('2018-10-01 00:00:01'), Timestamp('2018-11-20 00:00:00'))

**Conceptual Outline**:

Our key columns are userId, time and page. We want features that describe how the user behaved before the prediction point, not after, to prevent leakage from the future. 

Thus, we define a prediction time for each user. We want to predict churn in the next 10 days, thus we need a moment in time $T_\text{pred}$ at which we pretend to make the prediction. 

Then, there are two options. 1) The user churns (has a "cancellation confirmation" event), then we set $T_\text{pred}$ 10 days before $T_\text{churn}$. 2) The user does not churn in the training set, then we set the prediction time 10 days before the last observed event. $T_\text{pred} = T_\text{pred} - 10 \text{days}$.

Everythin we compute as features must only ever use events with time before $T_\text{pred}$, otherwise we have data leakage.

**Feature 1**:

I propose "recency of activity" , i.e. days since last event, as a feature predicting churn. Users who have not done anything for a long time are more likely to churn. For each user, we look at all events with timestamp before $T_\text{pred}$ and find the last event before $T_\text{pred}$.

A possible interpretation would be: Small value: The user was active right before the prediction point and has lower churn risk. Large value: The user was inactive for many days, higher churn risk.

**Feature 2:**

I propose short-term activity level as a predictor of churn. 

We define a seven day lookback window $[T_\text{pred} - 7 \text{days}, T_\text{pred}]$. Then, we count all events in this window and construct the feature "events_last_7d". 

Rationale: Many events in the last seven days before $T_\text{pred}$ indicates that the user likes the service, uses it regularly and thus is unlikely to churn. If the user barely uses the servide, he is more likely to cancel the subscription.

**Feature 2.1:**

A very similar feature with a seven days lookback window can be included with the sole difference that we only consider songs played as events in the window. In the end, users use the service to listen to songs and thus the number of songs played in the last seven days before prediction_ts might be a better indicator of churn inclination than the total amount of events. 

In [99]:
# create one function for feature engineering
def feature_engineering(df):
    # for churners only
    # compute churn timestamp per user
    churn_ts = (
        df[df['page'] == 'Cancellation Confirmation']
        .groupby('userId')['time']
        .min()
        .rename('churn_ts')
    )

    # last observed event per user
    last_ts = (
        df.groupby('userId')['time']
        .max()
        .rename('last_ts')
    )

    # registration time per user
    registration_ts = (
    df.groupby('userId')['registration']
    .min()
    .rename('registration_ts')
    )

    # combine in to user-level table
    user_meta = (
        pd.concat([last_ts, churn_ts, registration_ts], axis=1)
        .reset_index()  # keep userId as column
    )

    # create binary churn flag
    user_meta['churned'] = np.where(
        user_meta['churn_ts'].notnull(), 1, 0
    )

    # define the prediction time T_pred for each user
    lookahead = pd.Timedelta(days=10)

    def compute_prediction_ts(row):
        if pd.notnull(row['churn_ts']):
            # churner: predict ts is churn time
            return row['churn_ts']
        else:
            # non-churner: predict ts is last observation
            return row['last_ts']

    # define prediction timestamp
    user_meta['prediction_ts'] = user_meta.apply(compute_prediction_ts, axis=1)

    # define lookback windows for other features
    lookback_7d = pd.Timedelta(days=7)
    lookback_30d = pd.Timedelta(days=30)

    # compute the features per user

    # for convenience, set an index on user_meta
    user_meta_indexed = user_meta.set_index('userId')

    def make_features_for_user(user_df):
        # user_df holds all events for a given userId
        # shall return a series with feature values for this user
        user_id = user_df['userId'].iloc[0]
        pred_ts = user_meta_indexed.loc[user_id, 'prediction_ts']

        # only consider events before prediction time
        df_before_pred = user_df[user_df['time'] < pred_ts]

        # Feature 1: days_since_last_event
        if not df_before_pred.empty:
            last_event_ts = df_before_pred['time'].max()
            days_since_last_event = (pred_ts - last_event_ts).total_seconds() / (3600 * 24)
        else: 
            # no events before prediction time
            days_since_last_event = (pred_ts - registration_ts.loc[user_id]).total_seconds() / (3600 * 24)

        # Feature 2: events_last_7d
        window_start_7d = pred_ts - lookback_7d
        mask_7d = (df_before_pred['time'] >= window_start_7d) & (df_before_pred['time'] <= pred_ts)
        events_last_7d = df_before_pred[mask_7d].shape[0]

        # Feature 2.1 songs_last_7d
        songs_last_7d = df_before_pred.loc[mask_7d & (df_before_pred['page'] == 'NextSong')].shape[0]

        # Feature 3: number of sessions in last 7 days
        sessions_last_7d = df_before_pred.loc[mask_7d, 'sessionId'].nunique()

        # return the features as a pandas series
        features = pd.Series({
            'days_since_last_event': days_since_last_event,
            'events_last_7d': events_last_7d,
            'songs_last_7d': songs_last_7d,
            'sessions_last_7d': sessions_last_7d,
        })
        return features

    # pipeline to compute the features for each user
    features_df = (
        df.groupby('userId')
        .apply(make_features_for_user)
        .reset_index()
    )

    # merge with existing user base
    user_base = user_meta.copy()
    user_base = user_base.merge(features_df, on='userId', how='left')

    # return the user-level data
    return user_base

**First Logit Baseline:**

Our dataset has 19140 user, of which 4271 are churners (22%). Thus, the data is class-imbalanced and naive accuracy is not reliable. E.g., we can compute balanced accuracy as a performance measure. 

First, we must create a train/validation split since we are not allowed to touch the test data in any of our steps before the final model is decided on. The train/validation split enables us to train a model on 80% of the train set users and evaluate generalization on the remaining 20%. By observing the validation set performance, we can tune hyperparameters without touching the final test set. As our data is user-level aggregated and we assume the behavior of the users to be independent, a random split is tenable. All features were computed solely based on data before $T_\text{pred}$, so no time leakage occurs.

As a first baseline, we use a logistic regression classifier that models 

$P(\text{churn} = 1 | X) = \sigma ( \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ...)$

In our case we believe days_since_last_event to have a positive impact on churn probability since the users are inactive and events_last_7d and songs_last_7d to have negative coefficients since more recent interactions indicate satisfaction of the user and a lower likelihood to cancel the plan.

We use logistic regression as a first baseline since it is fast to estimate, easy to interpret and can provide calibrated probabilities. 

**Why use the balanced accuracy?**

Since we have 78% non-churners in our data, predicting non-churn for all users would yield an accuracy of 78% despite the model being worthless. The balanced accuracy is

$\text{Balanced Accuracy} = \frac{\text{TPR} + \text{TNR}}{2}$

Thus, it gives equal weight to predicting churners and non-churners and penalizes models that ignore the minority, i.e., churners in our case.

**Procedure:**

We select the features days_since_last_event, events_last_7d and songs_last_7d and the target vector churned. Then, we perform a train/validation split with val_size=0.2 and stratification. This means that the 22% churn proportion is preserved both in the test and the validation set. 

Then, we fit the LogisticRegression, optionally with scaled features or regularization but first in the most simple specification. Using this model, we predict the churn on the validation set and compute as performance metrics the accuracy and the balanced accuracy. 

**Export the user_base data**:

Now, we created a dataset that is user-aggregated and contains the different features without time leakage and the binary target variable "churned". We export this as a csv file such that we do not have to re-run the feature engineering code each time before fitting different classifier models.

In [57]:
# export user_base as csv
user_base.to_csv("user_base_cleaned.csv", index=False)

In [3]:
# read in the user_base data from the csv file
user_base = pd.read_csv("user_base_cleaned.csv")

In [100]:
user_base = feature_engineering(train)

C:\Users\juliu\AppData\Local\Temp\ipykernel_41092\3130954090.py:100: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(make_features_for_user)


In [101]:
user_base.head(5)

,userId,last_ts,churn_ts,registration_ts,churned,prediction_ts,days_since_last_event,events_last_7d,songs_last_7d,sessions_last_7d
0,1000025,2018-10-18 20:33:05,2018-10-18 20:33:05,2018-07-10 09:30:08,1,2018-10-18 20:33:05,0.001296,812.0,687.0,7.0
1,1000035,2018-11-15 03:53:11,NaT,2018-09-12 19:28:22,0,2018-11-15 03:53:11,0.000012,501.0,405.0,5.0
2,1000083,2018-10-12 10:04:58,2018-10-12 10:04:58,2018-09-07 18:01:49,1,2018-10-12 10:04:58,0.000197,476.0,406.0,5.0
3,1000103,2018-11-08 18:28:40,NaT,2018-09-22 07:27:25,0,2018-11-08 18:28:40,0.001088,6.0,5.0,1.0
4,1000164,2018-11-19 13:04:25,NaT,2018-08-12 09:32:01,0,2018-11-19 13:04:25,0.001528,383.0,313.0,4.0


In [102]:
print(user_base.shape)
print(user_base['churned'].value_counts())

(19140, 10)
churned
0    14869
1     4271
Name: count, dtype: int64


In [105]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, balanced_accuracy_score
# AutoML style implementation
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# select features and target variable
features = ['days_since_last_event', 'events_last_7d', 'songs_last_7d', 'sessions_last_7d']
target = 'churned'

X = user_base[features]
y = user_base[target]

# train validation split with stratification
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, 
    stratify=y,  # keeps churn proportion in both sets
    random_state=42
)
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Validation set size: {X_val.shape[0]} samples")

# use data and train-validation split from above
# define a pipeline
pipe = Pipeline([
    ('scaler', StandardScaler()),  # we'll allow passthrough
    ('clf', LogisticRegression())  # will be replaced
])

param_distributions = [
    # Logistic Regression with regularization
    {
        'scaler': [StandardScaler(), 'passthrough'],
        'clf': [LogisticRegression(max_iter=1000, class_weight='balanced')],
        'clf__C': np.logspace(-2, 2, 10),
        'clf__penalty': ['l2'],
        'clf__solver': ['lbfgs', 'liblinear']
    },
    # Random Forest
    {
        'scaler': ['passthrough'],  # trees do not need scaling
        'clf': [RandomForestClassifier(class_weight='balanced', random_state=42)],
        'clf__n_estimators': [100, 200, 300],
        'clf__max_depth': [None, 5, 10, 20],
        'clf__min_samples_split': [2, 5, 10],
        'clf__min_samples_leaf': [1, 2, 4]
    },
    # Gradient Boosting
    {
        'scaler': ['passthrough'],  # trees do not need scaling
        'clf': [GradientBoostingClassifier(random_state=42)],
        'clf__n_estimators': [100, 200, 300],
        'clf__learning_rate': [0.01, 0.05, 0.1],
        'clf__max_depth': [3, 5, 7]
    },
    # LightGBM
    {
        'scaler': ['passthrough'],  # trees do not need scaling
        'clf': [LGBMClassifier(objective='binary', boosting_type='gbdt', class_weight='balanced', random_state=42)],
        'clf__num_leaves': [15, 31, 63, 127],
        'clf__max_depth': [-1, 5, 8, 12],
        'clf__learning_rate': [0.01, 0.05, 0.1],
        'clf__n_estimators': [100, 200, 400],
        'clf__min_child_samples': [10, 20, 50],
        'clf__subsample': [0.6, 0.8, 1.0],
        'clf__colsample_bytree': [0.6, 0.8, 1.0]
    },
    # AdaBoost with Decision Tree base estimator
    {
        'scaler': ['passthrough'],  # trees do not need scaling
        'clf': [AdaBoostClassifier(
            estimator=DecisionTreeClassifier(class_weight='balanced', max_depth=1),
            random_state=42
        )],
        'clf__n_estimators': [50, 100, 200, 400],
        'clf__learning_rate': [0.01, 0.05, 0.1, 0.5, 1.0]
    }
]

# Configure AutoML Search
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_distributions,
    n_iter=50,  # number of parameter settings sampled
    scoring='balanced_accuracy',
    n_jobs=-1,
    cv=cv,
    verbose=2,
    random_state=42
)
# refit=True by default, the best hyperparameters are used to refit the model on the whole training set

# run the search
search.fit(X_train, y_train)

print("Best CV balanced acuracy:", search.best_score_)
print("Best parameters:")
for k, v in search.best_params_.items():
    print(f"  {k}: {v}")
best_model = search.best_estimator_

# predict the valdation set
y_pred = best_model.predict(X_val)

# performance metrics
acc = accuracy_score(y_val, y_pred)
bacc = balanced_accuracy_score(y_val, y_pred)

print("Classification report:")
print(classification_report(y_val, y_pred))
print(f"Accuracy: {acc:.4f}")
print(f"Balanced Accuracy: {bacc:.4f}")

Training set size: 15312 samples
Validation set size: 3828 samples
Fitting 5 folds for each of 50 candidates, totalling 250 fits
[LightGBM] [Info] Number of positive: 3417, number of negative: 11895
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000519 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 785
[LightGBM] [Info] Number of data points in the train set: 15312, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG

In [106]:
best_model

,steps,"[('scaler', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,boosting_type,'gbdt'
,num_leaves,63
,max_depth,5
,learning_rate,0.05
,n_estimators,200
,subsample_for_bin,200000
,objective,'binary'


**Test Estimation for Kaggle upload**

We run our best model on the so far unseen test data, compute the binary predictions and upload this attempt to kaggle.

In [107]:
# estimation for train
user_base_test = feature_engineering(test)

C:\Users\juliu\AppData\Local\Temp\ipykernel_41092\3130954090.py:100: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(make_features_for_user)


In [108]:
user_base_test.head(10)

,userId,last_ts,churn_ts,registration_ts,churned,prediction_ts,days_since_last_event,events_last_7d,songs_last_7d,sessions_last_7d
0,1000655,2018-11-15 18:14:51,NaT,2018-09-14 11:53:55,0,2018-11-15 18:14:51,0.001933,35.0,26.0,3.0
1,1000963,2018-11-18 00:08:19,NaT,2018-09-07 02:07:18,0,2018-11-18 00:08:19,0.000428,485.0,412.0,5.0
2,1001129,2018-11-18 22:56:10,NaT,2018-08-25 10:13:19,0,2018-11-18 22:56:10,0.001134,61.0,46.0,2.0
3,1001963,2018-11-19 20:39:31,NaT,2018-10-10 06:49:13,0,2018-11-19 20:39:31,0.000012,325.0,250.0,2.0
4,1002283,2018-11-19 18:17:18,NaT,2018-09-26 21:29:18,0,2018-11-19 18:17:18,0.000012,302.0,250.0,5.0
5,1002397,2018-10-12 17:40:04,NaT,2018-07-31 17:48:32,0,2018-10-12 17:40:04,0.000012,38.0,31.0,1.0
6,1002533,2018-11-17 18:49:29,NaT,2018-06-26 14:49:42,0,2018-11-17 18:49:29,0.002569,125.0,98.0,5.0
7,1002712,2018-11-17 05:19:06,NaT,2018-09-23 12:19:55,0,2018-11-17 05:19:06,0.000012,7.0,4.0,1.0
8,1002879,2018-11-15 05:49:29,NaT,2018-07-06 12:01:28,0,2018-11-15 05:49:29,0.000625,67.0,54.0,2.0
9,1003703,2018-11-17 00:15:38,NaT,2018-07-14 23:31:29,0,2018-11-17 00:15:38,0.003576,85.0,65.0,3.0


In [110]:
# select features 
features = ['days_since_last_event', 'events_last_7d', 'songs_last_7d', 'sessions_last_7d']
X = user_base_test[features]

# predict
y_pred = best_model.predict(X)

# create dataframe with userId and predictions
predictions_df = pd.DataFrame({
    'id': user_base_test['userId'],
    'target': y_pred
})

predictions_df.head(10)

,id,target
0,1000655,0
1,1000963,1
2,1001129,1
3,1001963,0
4,1002283,0
5,1002397,0
6,1002533,0
7,1002712,0
8,1002879,1
9,1003703,0


In [111]:
# export as csv
predictions_df.to_csv("churn_predictions.csv", index=False)

In [112]:
predictions_df.shape

(2904, 2)